In [1]:
from sage.crypto.sboxes import Ascon
from sage.geometry.polyhedron.constructor import Polyhedron
from sage.crypto.sbox import SBox
import random
from p_tqdm import p_map
from tqdm import tqdm
from collections import Counter

# Basic Sbox Properties

In [2]:
Ascon.differential_branch_number()

3

In [3]:
Ascon.linear_branch_number()

3

In [4]:
Ascon.differential_uniformity()

8

In [5]:
Ascon.maximal_linear_bias_absolute()

8

In [6]:
Ascon.boomerang_uniformity()

16

In [7]:
def frequency_count(frequency, table):
    count = 0
    for row in list(table):
        count += list(row).count(frequency)
    return count

In [8]:
print("DDT")
for freq in range(0, 9, 2):
    print(f"{freq} - {frequency_count(freq, Ascon.difference_distribution_table())}")
    
print("\nLAT")
for freq in range(-8, 9, 2):
    print(f"{freq} - {frequency_count(freq, Ascon.linear_approximation_table())}")
    
print("\nBCT")
for freq in range(0, 17, 2):
    print(f"{freq} - {frequency_count(freq, Ascon.boomerang_connectivity_table())}")

DDT
0 - 707
2 - 176
4 - 120
6 - 0
8 - 20

LAT
-8 - 18
-6 - 0
-4 - 174
-2 - 0
0 - 647
2 - 0
4 - 162
6 - 0
8 - 22

BCT
0 - 445
2 - 176
4 - 150
6 - 0
8 - 110
10 - 0
12 - 50
14 - 0
16 - 30


## 1-1 DDT, 1-1 LAT

In [9]:
def bit_1_1(sbox):
    if type(sbox) == list:
        sbox = SBox(sbox)
    elif type(sbox) != sage.crypto.sbox.SBox:
        raise TypeError("Sbox neither of type list nor sage.crypto.sbox.SBox")
        
    ddt = sbox.difference_distribution_table()
    lat = sbox.linear_approximation_table()
    bit_lat = []
    bit_ddt = []
    for i in range(sbox.m):
        bit_ddt.append([ddt[1 << i][1 << j] for j in range(sbox.m)])
        bit_lat.append([lat[1 << i][1 << j] for j in range(sbox.m)])
        
    return bit_ddt, bit_lat

In [10]:
bit_1_1(Ascon)

([[0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]])

## BDT

In [11]:
Ascon_inv = Ascon.inverse()

In [12]:
bdt = [[[0] * 32 for i in range(32)] for i in range(32)]


for diff_0 in range(32):
    for diff_1 in range(32):
        for back_diff_0 in range(32):
            for x in range(32):
                if Ascon_inv[Ascon[x] ^^ back_diff_0] ^^ Ascon_inv[Ascon[x ^^ diff_0] ^^ back_diff_0] == diff_0:
                    if Ascon[x] ^^ Ascon[x ^^ diff_0] == diff_1:
                        bdt[diff_0][diff_1][back_diff_0] += 1

In [13]:
count = Counter()
for row in bdt:
    for col in row:
        count.update(col)
count

Counter({0: 31624, 4: 720, 2: 352, 8: 40, 32: 32})

In [14]:
bct = Ascon.boomerang_connectivity_table()

def latex_table(table):
    print("|c|" + "c"*32 + "|")
    print(" & ".join([f"{i:x}" for i in range(32)]) + "\\\\")
    print("\\hline")
    for i, row in enumerate(list(table)):
        print(f"{i} & " + " & ".join([str(value) if value != 0 else "$\\cdot$" for value in list(row)]) + "\\\\")
    print("\\hline")

## Convex Hull

In [15]:
def generate_vertices(sbox):
    ddt = sbox.difference_distribution_table()
    vertices = []
    for row in range(2**sbox.m):
        for col in range(2**sbox.m):
            if ddt[row][col]:
                vertices.append(list(map(int, list(f"{bin(row)[2:].zfill(sbox.m)}{bin(col)[2:].zfill(sbox.m)}"))))
    return vertices

In [16]:
ascon_hull = list(Polyhedron(vertices=generate_vertices(Ascon)).inequality_generator())

ascon_hull

[An inequality (-1, 0, 0, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (0, -1, 0, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (0, 0, -1, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (0, 0, 0, -1, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (-1, 0, 0, -1, -1, 0, 0, -1, -1, 0) x + 4 >= 0,
 An inequality (0, -1, 0, -1, -1, 0, 1, 1, 1, 0) x + 2 >= 0,
 An inequality (0, 0, 0, 0, -1, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (1, -1, -1, -1, -1, 0, 1, 0, 0, 0) x + 3 >= 0,
 An inequality (0, 0, 0, 0, 0, 0, 1, 0, 0, 0) x + 0 >= 0,
 An inequality (-1, 0, 0, -1, -1, 0, 0, 1, 1, 0) x + 2 >= 0,
 An inequality (0, 0, -1, -1, -1, 0, 1, 1, 1, 0) x + 2 >= 0,
 An inequality (-3, -2, -3, -4, -6, -1, -3, -3, -3, -1) x + 23 >= 0,
 An inequality (-1, -1, -1, -1, -1, 0, -1, 0, 0, 0) x + 5 >= 0,
 An inequality (-1, -1, -1, -2, -2, 0, -1, -1, -1, 0) x + 8 >= 0,
 An inequality (-1, -1, -1, -1, 1, 0, 1, 0, 0, 0) x + 3 >= 0,
 An inequality (0, -1, -1, -1, -1, 0, 0, -1, -1, 0) x + 5 >= 0,
 An inequality

In [17]:
len(ascon_hull)

2415

## Sboxes - LuT, ANF, K-Map

In [18]:
def anf(s_box):
    result = []
    for i in range(0, s_box.m):
        result.append(
            [term.split("*") for term in str(s_box.component_function(1 << i).algebraic_normal_form()).split(" + ")]
        )
    return result

In [19]:
kmap = """y[0] = x[4] x[3] x[1]  + x[4]' x[3] x[1]'  + x[3]' x[1]' x[0] + x[3]' x[1] x[0]';
y[1] = x[4] x[3] x[2]  + x[4] x[3]' x[2]'  + x[3] x[2] x[1]' x[0] + x[3]' x[2]' x[1]' x[0] + x[3] x[2] x[1] x[0]' + x[3]' x[2]' x[1] x[0]' + x[4]' x[3]' x[2] x[1] x[0] + x[4]' x[3] x[2]' x[1] x[0] + x[4]' x[3]' x[2] x[1]' x[0]' + x[4]' x[3] x[2]' x[1]' x[0]';
y[2] = x[3] x[2] x[1]  + x[3]' x[2]' x[1]  + x[3] x[2] x[0]' + x[3]' x[2]' x[0]' + x[3]' x[2] x[1]' x[0] + x[3] x[2]' x[1]' x[0];
y[3] = x[4]' x[3] x[2] x[1] x[0] + x[4]' x[3]' x[2]' x[1]' x[0] + x[4] x[3] x[2] x[1] x[0]' + x[4] x[3]' x[2]' x[1]' x[0]' + x[4] x[3]' x[2] x[0] + x[4] x[2]' x[1] x[0] + x[4] x[3] x[1]' x[0] + x[4]' x[3]' x[2] x[0]' + x[4]' x[2]' x[1] x[0]' + x[4]' x[3] x[1]' x[0]';
y[4] = x[3] x[1] x[0] + x[3] x[1]' x[0]' + x[4] x[3]' x[2] x[1]  + x[4]' x[3]' x[2]' x[1]  + x[4]' x[3]' x[2] x[1]'  + x[4] x[3]' x[2]' x[1]';"""


In [20]:
def verilog_lut_sbox(sbox, export_file="sbox"):
    """
    Takes a n-bit S-Box as input and generates a Verilog script file
    implementing the S-Box.
    """
    # Define the Verilog module for the S-Box.
    module = f"""module {export_file}(input [{len(sbox) - 1}:0] x, output reg [{len(sbox) - 1}:0] y);
    always @(*)
        case (x)
"""
    for s_in in range(2 ** len(sbox)):
        module += f"\t\t{len(sbox)}'d{s_in}: y = {len(sbox)}'d{sbox[s_in]};\n"
    module += f"""
            default: y = 5'd0;
        endcase
endmodule"""
    
    # Write the Verilog module to a file.
    with open(f"{export_file}.v", "w") as f:
        f.write(module)
        
    print(f"Verilog S-Box module written to {export_file}.v")

In [21]:
def verilog_anf_sbox(anf, export_file="sbox"):
    """
    Takes a n-bit S-Box in SageMath lgebraic normal form as input and
    generates a Verilog script file implementing the S-Box in algebraic
    normal form.
    """
    # Define the Verilog module for the S-Box.
    module = f"""module {export_file}(input [{len(anf) - 1}:0] x, output reg [{len(anf) - 1}:0] y);
    always @(x)
    begin
    """

    # Generate the Verilog code for the S-Box.
    for y_bit in range(len(anf)):
        module += f"\ty[{y_bit}] <= "
        for term_index, term in enumerate(anf[y_bit]):
            if term_index == 0:
                module += "("
            else:
                module += ") ^ ("
            for x_index, x_bit in enumerate(term):
                if x_index == 0 and len(x_bit) == 2:
                    module += f"x[{x_bit[-1]}]"
                elif len(x_bit) == 2:
                    module += f" & x[{x_bit[-1]}]"
                else:
                    module += x_bit
            if term_index == len(anf[y_bit]) - 1:
                module += ");\n\t"

    module += "end\nendmodule"

    # Write the Verilog module to a file.
    with open(f"{export_file}.v", "w") as f:
        f.write(module)

    print(f"Verilog S-Box module in ANF written to {export_file}.v")

In [22]:
def verilog_kmap_sbox(logic, export_file="sbox"):
    lines = [line.split("=")[1].strip() for line in logic.split("\n") if line]
    # Define the Verilog module for the S-Box.
    module = f"""module {export_file}(input [{len(lines) - 1}:0] x, output [{len(lines) - 1}:0] y);
begin
"""
    # Generate the Verilog code for the S-Box.
    for y_bit, line in enumerate(lines):
        module += f"\tassign y[{y_bit}] = ("
        new_terms = []
        for term in [t.strip() for t in line.split("+") if t]:
            new_term = [x_bit if "'" not in x_bit else "!" + x_bit.replace("'", "") for x_bit in term.split()]
            new_terms.append(" & ".join(new_term))
        new_terms[-1] = new_terms[-1][:-1] + ");\n" 
        module += ") | (".join(new_terms)
    
    module += "end\nendmodule"

    # Write the Verilog module to a file.
    with open(f"{export_file}.v", "w") as f:
        f.write(module)

    print(f"Verilog S-Box module in KMAP written to {export_file}.v")


In [23]:
def generate_tcl(export_file="sbox"):
    script = f"""set_db lib_search_path ..
set_db library analysis.lib
set_db hdl_search_path /
read_hdl {export_file}.v
elaborate {export_file}
synthesize -to_mapped -effort medium
write_hdl > {export_file}_netlist.v
report gates > {export_file}.rep
exit"""

    # Write the tcl script to a file.
    with open(f"{export_file}.tcl", "w") as f:
        f.write(script)

    print(f"TCL Script written to {export_file}.tcl")

In [24]:
# # Generate Verilog Files for LUT and ANF
# verilog_lut_sbox(Ascon, "ascon_lut")
# verilog_anf_sbox(anf(Ascon), "ascon_anf")
# verilog_kmap_sbox(kmap, "ascon_kmap")

# # Generate TCL Script for Genus    
# generate_tcl("ascon_lut")
# generate_tcl("ascon_anf")
# generate_tcl("ascon_kmap")

generate_tcl("round_constant")
generate_tcl("linear")
generate_tcl("s_box_layer")
generate_tcl("permutation")

TCL Script written to round_constant.tcl
TCL Script written to linear.tcl
TCL Script written to s_box_layer.tcl
TCL Script written to permutation.tcl


# Zero - Sum

- Python Implementation of Ascon is based on - https://github.com/meichlseder/pyascon/
- Modified the Permutation - Removed the Round Constant Part as there is no use of it in Zero-Sum
- Added Permutation Inverse as per the Paper **IACR - 2016/490** and Generated a Verified Test-Vector
- Basic Zero Sum is implemented and is verified for any combination of backward and forward trails of upto a degree of 17
    - Forward - 1, Backward - 1, Degree - 4
    - Forward - 2, Backward - 1, Degree - 5
    - Forward - 3, Backward - 1, Degree - 9
    - Forward - 1, Backward - 2, Degree - 10
    - Forward - 2, Backward - 2, Degree - 10    
    - Forward - 3, Backward - 2, Degree - 10    
    - Forward - 4, Backward - 2, Degree - 17    

In [25]:
# === helper functions ===

def get_random_bytes(num):
    import os
    return to_bytes(os.urandom(num))

def zero_bytes(n):
    return n * b"\x00"

def to_bytes(l): # where l is a list or bytearray or bytes
    return bytes(bytearray(l))

def bytes_to_int(bytes):
    return sum([bi << ((len(bytes) - 1 - i)*8) for i, bi in enumerate(to_bytes(bytes))])

def bytes_to_state(bytes):
    return [bytes_to_int(bytes[8*w:8*(w+1)]) for w in range(5)]

def int_to_bytes(integer, nbytes):
    return to_bytes([(integer >> ((nbytes - 1 - i) * 8)) % 256 for i in range(nbytes)])

def rotr(val, r):
    return (val >> r) | ((val & (1<<r)-1) << (64-r))

def bytes_to_hex(b):
    return b.hex()
    #return "".join(x.encode('hex') for x in b)

def printstate(S, description=""):
    print(" " + description)
    print(" ".join(["{s:016x}".format(s=s) for s in S]))

def printwords(S, description=""):
    print(" " + description)
    print("\n".join(["  x{i}={s:016x}".format(**locals()) for i, s in enumerate(S)]))

    
def get_bin_words(S):
    return ["{s:064b}".format(**locals()) for s in S]

In [26]:
# ==== ASCON Permutation ====
def ascon_permutation(S, rounds=1, debug=False):
    """
    Ascon core permutation for the sponge construction
    S: Ascon state, a list of 5 64-bit integers
    rounds: number of rounds to perform
    returns nothing, updates S
    """
    assert(rounds <= 12)
    if debug: printwords(S, "Permutation input:")
    for r in range(rounds):
    # --- substitution layer ---
        S[0] ^^= S[4]
        S[4] ^^= S[3]
        S[2] ^^= S[1]
        T = [(S[i] ^^ 0xFFFFFFFFFFFFFFFF) & S[(i+1)%5] for i in range(5)]
        for i in range(5):
            S[i] ^^= T[(i+1)%5]
        S[1] ^^= S[0]
        S[0] ^^= S[4]
        S[3] ^^= S[2]
        S[2] ^^= 0XFFFFFFFFFFFFFFFF

        if debug: printwords(S, "Substitution layer:")
        # --- linear diffusion layer ---
        S[0] ^^= rotr(S[0], 19) ^^ rotr(S[0], 28)
        S[1] ^^= rotr(S[1], 61) ^^ rotr(S[1], 39)
        S[2] ^^= rotr(S[2],  1) ^^ rotr(S[2],  6)
        S[3] ^^= rotr(S[3], 10) ^^ rotr(S[3], 17)
        S[4] ^^= rotr(S[4],  7) ^^ rotr(S[4], 41)
        if debug: printwords(S, "Linear diffusion layer:")


def ascon_permutation_inv(S, rounds=1, debug=False):
    """
    Ascon core permutation inverse for the sponge construction
    S: Ascon state, a list of 5 64-bit integers
    rounds: number of rounds to perform
    returns nothing, updates S
    """
    assert(rounds <= 12)
    if debug: printwords(S, "Permutation input:")    
    # Linear Layer Inverse Rotations - Rivest
    rotations = [
        [0, 3, 6, 9, 11, 12, 14, 15, 17, 18, 19, 21, 22, 24, 25, 27, 30, 33, 36, 38, 39, 41, 42, 44, 45, 47, 50, 53, 57, 60, 63],
        [0, 1, 2, 3, 4, 8, 11, 13, 14, 16, 19, 21, 23, 24, 25, 27, 28, 29, 30, 35, 39, 43, 44, 45, 47, 48, 51, 53, 54, 55, 57, 60, 61],
        [0, 2, 4, 6, 7, 10, 11, 13, 14, 15, 17, 18, 20, 23, 26, 27, 28, 32, 34, 35, 36, 37, 40, 42, 46, 47, 52, 58, 59, 60, 61, 62, 63],
        [1, 2, 4, 6, 7, 9, 12, 17, 18, 21, 22, 23, 24, 26, 27, 28, 29, 31, 32, 33, 35, 36, 37, 40, 42, 44, 47, 48, 49, 53, 58, 61, 63],
        [0, 1, 2, 3, 4, 5, 9, 10, 11, 13, 16, 20, 21, 22, 24, 25, 28, 29, 30, 31, 35, 36, 40, 41, 44, 45, 46, 47, 48, 50, 53, 55, 60, 61, 63]
    ]
    
    for r in range(rounds):
         # --- linear diffusion layer ---
        
        for word, word_rot in enumerate(rotations):
            source_word = S[word]
            for i, rot in enumerate(word_rot):
                if i:
                    S[word] ^^= rotr(source_word, rot)
                else :
                    S[word] = rotr(source_word, rot)
                    
        if debug: printwords(S, "Inverse linear diffusion layer:")
        
        # --- substitution layer ---
        bin_words = get_bin_words(S)
        new_bin_words = ["" for i in range(5)]
        for bit in range(64):
            s_box_out = f"{Ascon.inverse()[int(bin_words[0][bit] + bin_words[1][bit] + bin_words[2][bit] + bin_words[3][bit] + bin_words[4][bit], 2)]:05b}"
            for word in range(5):
                new_bin_words[word] += s_box_out[word]
        for word in range(5):
            S[word] = int(new_bin_words[word], 2)
        if debug: printwords(S, "Inverse Substitution layer:")
        

In [27]:
S = [
    int( "80400c0600000000" , 16),
    int("c82cbe1c72be1a3a" , 16),
    int("85621d92797f8475", 16),
    int("23fd6519897d9e12", 16),
    int("5c0609b2f5ca3aaa" ,16)
]

ascon_permutation(S, 12, True)


ascon_permutation_inv(S, 12, True)

 Permutation input:
  x0=80400c0600000000
  x1=c82cbe1c72be1a3a
  x2=85621d92797f8475
  x3=23fd6519897d9e12
  x4=5c0609b2f5ca3aaa
 Substitution layer:
  x0=a6d7d29582081a47
  x1=3399fe3b0e09a4c5
  x2=eeb354d380bc4118
  x3=b2b5cf2177763af7
  x4=7fd3d6a37e83a4a8
 Linear diffusion layer:
  x0=851e82351527835e
  x1=d94a1caaf423b110
  x2=fa5033e90ee09090
  x3=1222bb0858bb5cc2
  x4=7e9330dc6c414a0a
 Substitution layer:
  x0=b56e167ec35def1c
  x1=10f71d0a87bd2486
  x2=b074d068217cdc77
  x3=cea5a4b6cf3cb65e
  x4=6cf387d6b0fb26c8
 Linear diffusion layer:
  x0=3d5371185773509b
  x1=82408f17b675ee8c
  x2=b68f6b1d9147413d
  x3=02396a8d30d41eed
  x4=17721d4a79aca946
 Substitution layer:
  x0=8ba5f898f6854947
  x1=9ede89c8896c062c
  x2=de720eb791e5f14c
  x3=099493d57849580a
  x4=950bf4c2d958b1ab
 Linear diffusion layer:
  x0=4ad99d94f3c9de18
  x1=f9381f809b318a5b
  x2=8232c1d687509e2f
  x3=a713f23bc4fdf678
  x4=a34d4f7389208532
 Substitution layer:
  x0=ffe0a2f9b264bc15
  x1=96bf2d182504275d
  x2=84

In [31]:
# TODO - Change bit_length to 5 64 bit words 

def generate_inputs(degree):
    if 320 < degree:
        raise ValueError(f"Given Degree {degree} is greater than 320 bits")
        
    constant= bin(random.randint(0, 2 ** (320 - degree)))[2:].zfill(320 - degree)
    variable_positions = sorted(random.sample(range(320), degree))
    
    inputs = []
    for var in tqdm(range(2 ** degree), desc="Generating Inputs"):
        S = [0, 0, 0, 0, 0]
        variable = bin(var)[2:].zfill(degree)
        input_bin = list(constant)
        for word in range(5):
            for index, position in enumerate(variable_positions):
                if position < len(input_bin):
                    input_bin.insert(position, variable[index])               
                elif variable_positions[index] >= variable_positions[index - 1]:
                    input_bin.append(variable[index])
            
#         pprint(["".join(input_bin[word * 64 : (word + 1) * 64 - 1]) for word in range(5)])
        inputs.append([int("".join(input_bin[word * 64 : (word + 1) * 64 - 1]) , 2) for word in range(5)])
        
    return inputs

In [34]:
def zero_sum(forward_rounds=3, backward_rounds=1):
    degree = max(3 ** backward_rounds, 2 ** forward_rounds) + 1
    print(f"Forward = {forward_rounds}, Backward = {backward_rounds}, Degree - max(2^forward, 3^backward) + 1 = {degree}")
    input_sum = [0, 0, 0, 0, 0]
    forward_sum = [0, 0, 0, 0, 0]
    backward_sum = [0, 0, 0, 0, 0]
    inputs = generate_inputs(degree)
    for input_val in tqdm(inputs, desc="Summing Inputs and Outputs"):
        backward_val = input_val[:]
        ascon_permutation_inv(backward_val, backward_rounds)
        
        for word in range(5):
            input_sum[word] ^^= input_val[word]
            
        ascon_permutation(input_val, forward_rounds)
        for word in range(5):
            forward_sum[word] ^^=  input_val[word]
            backward_sum[word] ^^=  backward_val[word]
    
    print(f"Order - {degree}, Input Sum - {input_sum}, Forward Sum - {forward_sum}, Forward Sum - {backward_sum} ")

In [37]:
zero_sum(4, 2)

Generating Inputs:   6%|███▎                                                   | 7811/131072 [00:00<00:03, 39136.88it/s]

Forward = 4, Backward = 2, Degree - max(2^forward, 3^backward) + 1 = 17


Summing Inputs and Outputs: 100%|██████████████████████████████████████████████| 131072/131072 [17:10<00:00, 127.19it/s]

Order - 17, Input Sum - [0, 0, 0, 0, 0], Forward Sum - [0, 0, 0, 0, 0], Forward Sum - [0, 0, 0, 0, 0] 
